<a href="https://colab.research.google.com/github/13194307/UTS_ML2019_ID13194307/blob/master/ML_A2/NeuralNetwork.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import math
import numpy as np
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from scipy.special import softmax
from scipy import stats

In [0]:
class NeuralNetwork:
    class Layer:
        class Neuron:
            def __init__(self, inputShape):
                self.weights, self.bias = self.initialiseWeights(inputShape)
                
            def initialiseWeights(self, inputShape):
                weights = np.array([np.random.randn() for _ in range(0, inputShape)]) / math.sqrt(inputShape)
                bias = np.random.randn()
                
                return weights, bias
            
            def getWeightsAndBias(self):
                return self.weights, self.bias
            
            def generateNeuronOutput(self, x):
                return np.dot(self.weights, x) + self.bias
            
            
            
        
        def __init__(self, layerType, neuronsPerLayer, inputShape):
            self.layerType = layerType
            self.numNeurons = neuronsPerLayer
            self.inputShape = inputShape
            self.neurons = [self.Neuron(inputShape) for _ in range(0, neuronsPerLayer)]
            
        def getInputShape(self):
            return self.inputShape
        
        def getNumNeurons(self):
            return self.numNeurons
        
        def generateLayerOutput(self, x):
            layerOutput = []
            
            for neuron in self.neurons:
                neuronOutput = neuron.generateNeuronOutput(x)
                layerOutput.append(neuronOutput)
               
            #print(layerOutput)
            if self.layerType == "Dense":
                #ReLU activation
                layerOutput = np.maximum(layerOutput, 0)
            elif self.layerType == "Output":
                layerOutput = 1 / (1 + math.exp(-1*layerOutput[0]))
                #Softmax activation
                #print(layerOutput)
                #layerOutput = np.exp(layerOutput)/sum(np.exp(layerOutput))
            else:
                raise NotImplementedError
                    
            return layerOutput
            
        def __str__(self):
            output = ""
            for neuron in self.neurons:
                weights, bias = neuron.getWeightsAndBias()
                output+="\t"
                
                for j in range(0, len(weights)):
                    output+=("w{}: {}, ".format(j, weights[j]))
                    
                output+=("b0: {}\n".format(bias))
            
            return output
        
        
        
        
        
    
    def __init__(self):
        self.layers = []
        self.numLayers = 0
        
    def addLayer(self, layerType, neuronsPerLayer, inputShape=None):
        if inputShape == None:
            inputShape = self.layers[-1].getNumNeurons()
            
        self.layers.append(self.Layer(layerType, neuronsPerLayer, inputShape))
        self.numLayers+=1
        
    def predict(self, x):
        prediction = []
        
        for sampleX in x:
            pred = self.feedForward(sampleX)
            #print(pred)
            prediction.append(pred)
      
        prediction = np.array(prediction)
        #print(prediction)
        #prediction = softmax(prediction)
        #print(prediction)
        prediction[prediction > 0.5] = 1
        prediction[prediction <= 0.5] = 0
        return prediction
    
    def feedForward(self, x):
        lastLayerOutput = x
        
        for layer in self.layers:
            lastLayerOutput = layer.generateLayerOutput(lastLayerOutput)
            
        return lastLayerOutput
    
    def softmax(self, x): 
        """Compute softmax values for each sets of scores in x.""" 

        e_x = np.exp(x - np.max(x)) 

        return e_x / e_x.sum(axis=0) 

        # only difference
        
    def __str__(self):
        output = ""
        
        for i in range(0, self.numLayers):
            output+=("Layer {}:\n".format(i+1))
            output+=str(self.layers[i])
        
        return output

In [200]:
nn = NeuralNetwork()
nn.addLayer("Dense", 3, inputShape=4)
nn.addLayer("Dense", 3)
nn.addLayer("Dense", 3)
nn.addLayer("Output", 1)
print(nn)

Layer 1:
	w0: 0.23795143630272864, w1: -0.40157947893520496, w2: 0.9667570445752864, w3: 0.3464894017838772, b0: -0.990269908081075
	w0: 1.2057983733547877, w1: 0.0032066647947672335, w2: -0.1630241899405391, w3: -0.4895011618074342, b0: 1.310639937361324
	w0: 0.4976200108852722, w1: -0.16288868351125418, w2: 0.10906928530296535, w3: 0.5306781839393346, b0: 0.3697303918278565
Layer 2:
	w0: 0.12196677001087997, w1: -0.6930418425458341, w2: 0.06470342257724876, b0: -0.6216387976850688
	w0: 0.2210988659682155, w1: 0.036382628764374494, w2: -0.2803129512051275, b0: 1.806560703542133
	w0: 0.13880406514241345, w1: 0.5790437695253757, w2: 0.5253856387461914, b0: -1.5082299209762575
Layer 3:
	w0: -0.6260244842037828, w1: 0.1311912025512874, w2: -0.8501278219041793, b0: 0.3400664388249166
	w0: -1.5161681778415141, w1: 0.4334792183225777, w2: 0.07633195498906958, b0: -0.6587810878645449
	w0: -0.13823219585409957, w1: 0.01814875783861039, w2: 0.47906492161955966, b0: 0.5609435641359658
Layer 4:
	

In [0]:
from sklearn.datasets import load_iris

iris_X, iris_y = load_iris(True)

In [0]:
from sklearn.preprocessing import MinMaxScaler

iris_X_trimmed = iris_X

scaler = MinMaxScaler()
iris_X_scaled = scaler.fit_transform(iris_X_trimmed)
iris_X_zscore = stats.zscore(iris_X_trimmed)

In [201]:
pred = nn.predict(iris_X_zscore)
accuracy_score(pred, iris_y)

0.3333333333333333